In [1]:
import openai
import pandas as pd
import os
from dotenv import load_dotenv
# new
from openai import AsyncOpenAI
import base64


# year = 2021

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
# Load the environment variables from .env file
load_dotenv()

# Retrieve the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


# Configuration
openai.api_key = openai_api_key
model_name = 'gpt-4o'  # Ensure this model version supports sessions

prompt = ('You are a classifier. In the following, I am going to provide a figure from a peer-reviewed paper, and I want you to classify it according to the following '
              + 'categories: \n\nCategories: '+ 
              '\"process diagram\", \"conceptual diagram\", \"cycle\", \"hierarchical diagram\", \"bar chart\", ' + 
              '\"line graph\", \"scatter plot\", \"mixed statistical plot (more than 1 statistical plot type)\", \"data structure\",' + 
              ' \"data collection, data analysis, data gathering diagrams\", \"heatmap\", \"data map\", \"organizational chart\", \"timeline\"' + 
              '\n\nPlease choose the category that best represents, and most specifically describes, the figure type. Your response should only be an exact string matching one of the categories. '
              )

client = AsyncOpenAI()

for year in range(2000, 2021):
   
    # Load the Excel file into a DataFrame
    cropped_and_labeled_df = pd.read_excel(f"../cropped_and_labeled_image_data/{year}_cropped_and_labeled_figs.xlsx")

    count = 0
    # Loop through rows and check for 'none selected'
    for index, row in cropped_and_labeled_df.iterrows():
        if row["subcategory"] == "none selected":
            base64_image = encode_image("../" + row["new image path"])
            print(row["new image path"])
            image_completion = await client.chat.completions.create(model="gpt-4o", messages=[{"role": "user","content": [    
                {
                    "type": "text", 
                    "text": prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    },
                    
                }]}])
            subcategory = image_completion.choices[0].message.content.replace('"', '')
            print(subcategory)
            cropped_and_labeled_df.iloc[index, cropped_and_labeled_df.columns.get_loc('subcategory')] = subcategory
            count = count + 1

    print(f'Images classified: {count}')
    cropped_and_labeled_df.to_excel(f"../cropped_and_labeled_image_data/{year}_cropped_and_labeled_figs.xlsx", index=False)

final_figures/2000/F1_P1_Bansal & Roth_2000_AMJ_Why Companies Go Green.png
conceptual diagram
final_figures/2000/F2_P12_Bansal & Roth_2000_AMJ_Why Companies Go Green.png
conceptual diagram
final_figures/2000/F2_P15_Maznevski & Chudoba_2000_OrgSci_Bridging Qualitative and Quantitative Methods.png
line graph
final_figures/2000/F6_P15_Maznevski & Chudoba_2000_OrgSci_Bridging Space over Time.png
conceptual diagram
final_figures/2000/F1_P7_Orlikowski_2000_OrgSci_Using Technology and Constituting Structures.png
conceptual diagram
Images classified: 5
final_figures/2001/F2_P6_Thomas et al._2001_OrgSci_Understanding Strategic Learning.png
data structure
Images classified: 1
final_figures/2002/F1_P5_Beyer & Hannah_2002_OrgSci_Building on the past - Enacting Established Personal Identities in a New Work Setting.png
process diagram
final_figures/2002/F1_P5_Dutton et al._2002_Org Sci_Red Light Green Light.png
conceptual diagram
final_figures/2002/F1_P2_Huy_2002_ASQ_Emotional Balancing of Organizat